In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/Abjad_books.csv')

In [4]:
df.head()

,BookBadge_Title,Author,Rating,Description,Raters,Reviews,Readers,Link
0,ماجدولين,مصطفى لطفي المنفلوطي,4.4,لا نعرف عن الحُب غير وجهه الأليف الذي نُحِب، ل...,4057,4313,23055,https://www.abjjad.com/book/15445318/%D9%85%D8...
1,هكذا خُلقت,محمد حسين هيكل,4.2,روايةٌ واقعيةٌ تركَتْها بين يدَيِ المؤلف امرأة...,2701,2944,14769,https://www.abjjad.com/book/2019491840/%D9%87%...
2,قواعد العشق الأربعون,إليف شافاق,4.3,بلغت بطلة الرواية، إيلاّ الزوجة التعيسة، سنّ ا...,2643,2777,8242,https://www.abjjad.com/book/2138898432/%D9%82%...
3,في قلبي أنثى عبرية,خولة حمدي,4.5,في قلب حارة اليهود في الجنوب التونسي تتشابك ال...,3405,3518,10091,https://www.abjjad.com/book/2073100288/%D9%81%...
4,الأرواح المتمردة,جبران خليل جبران,4.3,«الأرواح المتمردة» هو كتاب صدر لأول مرة في مدي...,2478,2635,9857,https://www.abjjad.com/book/15445260/%D8%A7%D9...


In [ ]:
# نريد تدريب نموذج يقرأ وصف كتاب باللغة العربية، ويقوم بتصنيفه إلى تقييم منخفض أو متوسط أو عالي، بالاعتماد على المواضيع الكامنة داخل النصوص.

# نريد بناء نظام يستطيع أن:

# يقرأ أوصاف الكتب (وصف نصي).

# يستخرج منها المواضيع باستخدام LDA.

# يستخدم تلك المواضيع لتصنيف الكتاب حسب تقييمه إلى:

# 🟥 منخفض (أقل من 3.5)

# 🟨 متوسط (من 3.5 إلى أقل من 4.25)

# 🟩 عالي (4.25 أو أكثر)



In [5]:
df.shape

(440, 8)

In [ ]:
# تحويل التقييم إلى فئة تصنيفية
# لأن Rating رقمي (مثل 4.4 أو 3.2)، نحوله إلى فئات:

def classify_rating(r):
    if r < 3.5:
        return "منخفض"
    elif r < 4.25:
        return "متوسط"
    else:
        return "عالي"

df['RatingClass'] = df['Rating'].apply(classify_rating)


In [ ]:
# !pip install pandas nltk scikit-learn gensim pyarabic

In [6]:
descriptions = df['Description'].dropna().tolist()

In [7]:
import nltk
from nltk.corpus import stopwords
from pyarabic.araby import strip_tashkeel, strip_diacritics, normalize_hamza
import re

nltk.download('stopwords')
arabic_stopwords = set(stopwords.words("arabic"))

def preprocess(text):
    # إزالة التشكيل والنصوص غير العربية
    text = strip_diacritics(strip_tashkeel(normalize_hamza(text)))
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # إزالة الرموز غير العربية
    tokens = text.split()
    tokens = [t for t in tokens if t not in arabic_stopwords and len(t) > 2]
    return tokens

processed_descriptions = [preprocess(desc) for desc in descriptions]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Dictionary: تبني قاموس من الكلمات.

# Corpus: تحويل النصوص إلى قوائم bow (Bag-of-Words).

# LDA: تستخرج المواضيع الكامنة.

In [ ]:
#  إنشاء القاموس والكوربوس:
#  يقوم بإنشاء قاموس (dictionary) يحتوي على جميع الكلمات الفريدة الموجودة في قائمة texts.

# كل كلمة تأخذ معرّف رقمي (id).

# مثال:
# إذا كان لدينا texts = [["حب", "موت"], ["حب", "أمل"]]
# فإن dictionary.token2id يكون مثل: {"حب": 0, "موت": 1, "أمل": 2}

# corpus = [dictionary.doc2bow(text) for text in texts]
# يحوّل كل نص في texts إلى تمثيل بطريقة Bag of Words (BoW).

# كل نص يصبح قائمة من tuples (word_id, count).

# 🧠 مثال توضيحي:
# إذا كان text = ["حب", "أمل"]، يصبح:
# [(0, 1), (2, 1)]  # 0 = "حب", 2 = "أمل"

#  تدريب نموذج LDA:
#  هذا السطر يقوم بتدريب نموذج LDA (Latent Dirichlet Allocation).

# الهدف هو تعلم عدد 5 مواضيع (يمكنك تغييره).

# كل موضوع هو توزيع احتمالي على الكلمات، وكل نص هو توزيع على المواضيع.

# 🧠 مثلاً:

# موضوع 1 ← {"حب": 0.3, "موت": 0.2, "حرب": 0.1, ...}

# موضوع 2 ← {"إيمان": 0.4, "روح": 0.3, ...}
# دالة تحويل وصف كتاب إلى توزيع مواضيع:
# def get_topic_vector(bow):
#     topics = lda.get_document_topics(bow, minimum_probability=0)
#     return [prob for _, prob in sorted(topics, key=lambda x: x[0])]

# شرح:

# تأخذ وصف كتاب تم تحويله إلى BoW.

# تستخرج منه توزيعًا احتماليًا على المواضيع.

# تُرجع قائمة بطول num_topics، كل عنصر فيها يمثل احتمالية انتماء النص لكل موضوع.

# 🧠 مثال:
# [(0, 0.05), (1, 0.70), (2, 0.10), (3, 0.10), (4, 0.05)]
# ⇒ [0.05, 0.70, 0.10, 0.10, 0.05]

# تطبيق الدالة على كل النصوص للحصول على X:
# X = np.array([get_topic_vector(bow) for bow in corpus])

# لكل نص في corpus، يتم حساب توزيع المواضيع.

# النتيجة X هي مصفوفة (عدد الكتب × عدد المواضيع).

# هذه المصفوفة يمكن استخدامها كـ Features في أي نموذج تصنيف.

# 🧠 مثال تخيلي (5 مواضيع × 3 كتب):
# X =
# [[0.05, 0.70, 0.10, 0.10, 0.05],
#  [0.60, 0.10, 0.05, 0.15, 0.10],
#  [0.20, 0.20, 0.20, 0.20, 0.20]]

#  ماذا نحتاج هذه الخطوة؟
# لأننا نحتاج أن نحول نصوص غير منظمة (Unstructured) إلى بيانات عددية منظمة يمكن لنموذج التعلم الآلي (مثل XGBoost) فهمها.

In [8]:
#تحويل الكلمات إلى نموذج عددي (Bag of Words أو TF-IDF)
from gensim import corpora

# إنشاء القاموس
dictionary = corpora.Dictionary(processed_descriptions)

# تحويل النصوص إلى تمثيل عددي
corpus = [dictionary.doc2bow(text) for text in processed_descriptions]

In [9]:
# تطبيق LDA - Latent Dirichlet Allocation
from gensim.models import LdaModel

lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=5,  # عدد المواضيع المطلوب استخلاصها
                     random_state=42,
                     passes=10)

# طباعة المواضيع المستخلصة
topics = lda_model.print_topics(num_words=5)
for idx, topic in topics:
    print(f"🟢 موضوع {idx + 1}: {topic}")

🟢 موضوع 1: 0.009*"ءلى" + 0.008*"الكتاب" + 0.007*"كتاب" + 0.005*"النفس" + 0.005*"ءحد"
🟢 موضوع 2: 0.014*"الكتاب" + 0.009*"ءلى" + 0.004*"العالم" + 0.003*"ءكثر" + 0.003*"الكاتب"
🟢 موضوع 3: 0.009*"ءلى" + 0.009*"الكتاب" + 0.006*"لكل" + 0.006*"كتاب" + 0.006*"محمد"
🟢 موضوع 4: 0.014*"الكتاب" + 0.007*"ءلى" + 0.004*"وهي" + 0.003*"العالم" + 0.003*"يكون"
🟢 موضوع 5: 0.007*"ءلى" + 0.006*"الكتاب" + 0.005*"كتاب" + 0.004*"النفس" + 0.004*"كانت"


In [10]:
# !pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 87.2 MB/s eta 0:00:00


In [11]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.201109 -0.032197       1        1  21.964788
2      0.024038 -0.041717       2        1  20.542464
3     -0.140348 -0.102954       3        1  20.093022
1     -0.044471 -0.038824       4        1  19.049536
4     -0.040327  0.215692       5        1  18.350190, topic_info=        Term       Freq      Total Category  logprob  loglift
516    النفس  26.000000  26.000000  Default  30.0000  30.0000
446    ءحداث  13.000000  13.000000  Default  29.0000  29.0000
1510     لكل  20.000000  20.000000  Default  28.0000  28.0000
1157    رحلة   8.000000   8.000000  Default  27.0000  27.0000
549     كانت  18.000000  18.000000  Default  26.0000  26.0000
...      ...        ...        ...      ...      ...      ...
1020  الكثير   4.662110  11.404466   Topic5  -6.0822   0.8010
580    تاريخ   4.655135  11.408932   Topic5  -6.0837   0.7991
831    الفرد   4.654054   6.792109   Topic5  -6.0840   1.3175
846    يحاول   4.653835   6.792106   Topic5  -6.0840   1.3175
836      شيء   4.648913   9.852627   Topic5  -6.0851   0.9444

[279 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
829       5  0.950133  ءءليتها،
244       1  0.570631       ءحد
244       2  0.103751       ءحد
244       3  0.103751       ءحد
244       4  0.259378       ءحد
...     ...       ...       ...
1273      4  0.153807      يكون
871       1  0.690249      يمكن
871       2  0.172562      يمكن
871       5  0.172562      يمكن
338       4  0.791328      ينزل

[359 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2, 5])

In [12]:
import numpy as np

def get_topic_distribution(lda_model, bow, num_topics):
    topic_dist = lda_model.get_document_topics(bow, minimum_probability=0)
    return [prob for _, prob in sorted(topic_dist, key=lambda x: x[0])]

topic_vectors = [get_topic_distribution(lda_model, doc, num_topics=5) for doc in corpus]
X = np.array(topic_vectors)

In [13]:
y = df['Author']  # أو df['Rating'].astype(int)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

                            precision    recall  f1-score   support

           أحمد خالد توفيق       1.00      1.00      1.00         1
                 أحمد عيسى       1.00      1.00      1.00         1
                أسامة حمدي       1.00      1.00      1.00         1
    أسامة زيد وهبة الصيادي       1.00      1.00      1.00         2
              ألان م. ويبر       1.00      1.00      1.00         1
                   ألن ديب       1.00      1.00      1.00         1
           أمنية محمد لطفي       1.00      1.00      1.00         1
              أنطونان أرتو       1.00      1.00      1.00         1
                 أوسم وصفي       1.00      1.00      1.00         2
إبراهيم عبد القادر المازني       1.00      1.00      1.00         1
                إدورد برجر       1.00      1.00      1.00         1
              إيهاب حمارنة       1.00      1.00      1.00         1
              ا.س. رابوبرت       0.00      0.00      0.00         0
               باولو كويلو       0.00      0.00

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [19]:
# بناء القاموس والكوربوس
dictionary = corpora.Dictionary(processed_descriptions)
corpus = [dictionary.doc2bow(text) for text in processed_descriptions]

# تدريب نموذج LDA
lda = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10, random_state=42)

# تحويل كل وصف إلى تمثيل مواضيع
def get_topic_vector(bow):
    topics = lda.get_document_topics(bow, minimum_probability=0)
    return [prob for _, prob in sorted(topics, key=lambda x: x[0])]

X = np.array([get_topic_vector(bow) for bow in corpus])
y = df['Author']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

                            precision    recall  f1-score   support

           أحمد خالد توفيق       1.00      1.00      1.00         1
                 أحمد عيسى       1.00      1.00      1.00         1
                أسامة حمدي       1.00      1.00      1.00         1
    أسامة زيد وهبة الصيادي       1.00      1.00      1.00         2
              ألان م. ويبر       1.00      1.00      1.00         1
                   ألن ديب       1.00      1.00      1.00         1
           أمنية محمد لطفي       1.00      1.00      1.00         1
              أنطونان أرتو       1.00      1.00      1.00         1
                 أوسم وصفي       1.00      1.00      1.00         2
إبراهيم عبد القادر المازني       1.00      1.00      1.00         1
                إدورد برجر       1.00      1.00      1.00         1
              إيهاب حمارنة       1.00      1.00      1.00         1
              ا.س. رابوبرت       0.00      0.00      0.00         0
               باولو كويلو       0.00      0.00

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [21]:
def predict_new_description(text):
    tokens = preprocess(text)
    bow = dictionary.doc2bow(tokens)
    topic_vector = get_topic_vector(bow)
    prediction = clf.predict([topic_vector])[0]
    return prediction

new_desc = "رواية تتحدث عن الحب والحرب في زمن مضطرب، ومشاعر امرأة تعيش صراعًا داخليًا..."
author_prediction = predict_new_description(new_desc)
print("المؤلف المتوقع:", author_prediction)

المؤلف المتوقع: أدهم شرقاوي


In [24]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from pyarabic.araby import strip_tashkeel, strip_diacritics, normalize_hamza
from gensim import corpora
from gensim.models import LdaModel
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
from sklearn.preprocessing import LabelEncoder

# تحميل كلمات الوقف
nltk.download('stopwords')
arabic_stopwords = set(stopwords.words("arabic"))
# تحويل الفئات النصية إلى أرقام باستخدام LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['RatingClass'])  # الآن y ستكون أرقامًا: 0، 1، 2


df = df.dropna(subset=["Description", "Rating"])  # نستخدم Rating هنا

# تصنيف التقييمات إلى فئات (مثلاً: منخفض، متوسط، عالي)
def classify_rating(r):
    if r < 3.5:
        return "منخفض"
    elif r < 4.25:
        return "متوسط"
    else:
        return "عالي"

df['RatingClass'] = df['Rating'].apply(classify_rating)

# المعالجة المسبقة للنصوص
def preprocess(text):
    text = strip_diacritics(strip_tashkeel(normalize_hamza(text)))
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    tokens = text.split()
    return [t for t in tokens if t not in arabic_stopwords and len(t) > 2]

texts = [preprocess(desc) for desc in df['Description']]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# تدريب LDA
lda = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10, random_state=42)

# تحويل كل وصف إلى توزيع مواضيع
# حويل كل وصف إلى تمثيل رقمي للمواضيع
# كل كتاب يتحول إلى 5 أرقام تمثل احتمالية انتمائه لكل موضوع:
def get_topic_vector(bow):
    topics = lda.get_document_topics(bow, minimum_probability=0)
    return [prob for _, prob in sorted(topics, key=lambda x: x[0])]

X = np.array([get_topic_vector(bow) for bow in corpus])
y = y_encoded

# تدريب المصنف باستخدام XGBoost
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

print(classification_report(y_test_labels, y_pred_labels))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

        عالي       0.90      0.88      0.89        49
       متوسط       0.82      0.86      0.84        37
       منخفض       1.00      0.50      0.67         2

    accuracy                           0.86        88
   macro avg       0.91      0.75      0.80        88
weighted avg       0.87      0.86      0.86        88



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:15:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [25]:
def predict_rating_class(desc):
    tokens = preprocess(desc)
    bow = dictionary.doc2bow(tokens)
    topic_vec = get_topic_vector(bow)
    pred_num = xgb_clf.predict([topic_vec])[0]
    pred_label = label_encoder.inverse_transform([pred_num])[0]
    return pred_label

new_desc = "رواية تتحدث عن الحب والحرب في زمن مضطرب، ومشاعر امرأة تعيش صراعًا داخليًا..."
author_prediction = predict_new_description(new_desc)
print("المؤلف المتوقع:", author_prediction)

المؤلف المتوقع: أدهم شرقاوي


In [26]:
import joblib

# تخزين المصنف
joblib.dump(xgb_clf, "xgb_rating_classifier.pkl")

# تخزين LDA والقاموس
lda.save("lda_model.model")
dictionary.save("lda_dictionary.dict")
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [27]:
# تحميل النماذج المحفوظة
xgb_clf = joblib.load("xgb_rating_classifier.pkl")
lda = LdaModel.load("lda_model.model")
dictionary = corpora.Dictionary.load("lda_dictionary.dict")
label_encoder = joblib.load("label_encoder.pkl")

def predict_rating_class(desc):
    tokens = preprocess(desc)
    bow = dictionary.doc2bow(tokens)
    topic_vec = get_topic_vector(bow)
    pred_num = xgb_clf.predict([topic_vec])[0]
    pred_label = label_encoder.inverse_transform([pred_num])[0]
    return pred_label

# تجربة:
text = "رواية روحانية تبحث عن الذات وتناقش الحب والخسارة في زمن الحرب"
print("تقييم متوقع:", predict_rating_class(text))


تقييم متوقع: عالي


In [28]:
import joblib

# حفظ المصنف
joblib.dump(xgb_clf, "xgb_rating_classifier.pkl")

# حفظ LDA والقاموس
lda.save("lda_model.model")
dictionary.save("lda_dictionary.dict")
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']